<a href="https://colab.research.google.com/github/dahlia52/Commercial-District-Analysis/blob/main/code/%EC%83%81%EA%B6%8C_%EB%B6%84%EC%84%9D%EC%9D%84_%EC%9C%84%ED%95%9C_%ED%91%9C%EC%A4%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## 상권 분석을 위한 표준 데이터 정리

### 시군구, 행정동, 법정동 컬럼 생성

In [ ]:
local = pd.read_csv('/content/drive/MyDrive/광진구팟/Data/상권분석/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949')
local.drop(['형태정보','기준_년월_코드'],axis=1, inplace=True)
local.rename(columns = {'행정동_코드':'행정동코드'}, inplace=True)
local.head()

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동코드
0,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710
1,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611
2,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680
3,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690
4,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562


In [ ]:
local.shape

(1671, 8)

In [ ]:
dong = pd.read_csv('/content/drive/MyDrive/광진구팟/Data/상권분석/전국 행정동 코드 .csv')
dong.rename(columns = {'읍면동명':'행정동명'}, inplace = True)
dong.head()

,행정동코드,시군구명,행정동명
0,11000000,서울특별시,NaN
1,11110000,종로구,NaN
2,11110515,종로구,청운효자동
3,11110530,종로구,사직동
4,11110540,종로구,삼청동


In [ ]:
dong.dropna(inplace = True)

In [ ]:
standard = pd.merge(local, dong, on = '행정동코드', how = 'left')

In [ ]:
standard.shape

(1671, 10)

In [ ]:
standard.head()

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동코드,시군구명,행정동명
0,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710,금천구,시흥제5동
1,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611,노원구,하계1동
2,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680,금천구,시흥제2동
3,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690,금천구,시흥제3동
4,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562,송파구,방이2동


In [ ]:
standard.isnull().sum() # NULL 없음 - 일대일대응 완료

상권_구분_코드       0
상권_구분_코드_명     0
상권_코드          0
상권_코드_명        0
엑스좌표_값         0
와이좌표_값         0
시군구_코드         0
행정동코드          0
시군구명          29
행정동명          29
dtype: int64

### 위도, 경도 생성

In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 위도, 경도 열 생성
import pyproj

def project_array(coord, p1_type, p2_type):
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [ ]:
coord = np.array(standard[['엑스좌표_값', '와이좌표_값']])

# 좌표계 정보 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

/usr/local/lib/python3.9/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.9/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-76-b6a794f94413>:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[

array([[126.90674065,  37.45143757],
       [127.06887733,  37.6372189 ],
       [126.91087431,  37.4537563 ],
       ...,
       [127.11317658,  37.51922708],
       [127.0579595 ,  37.51375363],
       [126.99227486,  37.53718521]])

In [ ]:
standard['경도'] = result[:,0]
standard['위도'] = result[:,1]

### 최종 표준 데이터 정리 및 저장

In [ ]:
# 열 순서 정리
standard = standard[['시군구명', '행정동명', '상권_구분_코드_명', '상권_코드_명', '경도','위도','엑스좌표_값', '와이좌표_값','행정동코드','상권_코드', '상권_구분_코드']]
standard.head()

,시군구명,행정동명,상권_구분_코드_명,상권_코드_명,경도,위도,엑스좌표_값,와이좌표_값,행정동코드,상권_코드,상권_구분_코드
0,금천구,시흥제5동,골목상권,시흥동 은행나무사거리,126.906741,37.451438,191934,438813,11545710,2110753,A
1,노원구,하계1동,골목상권,노원 을지대학교병원,127.068877,37.637219,206265,459431,11350611,2110427,A
2,금천구,시흥제2동,골목상권,동일여자고등학교,126.910874,37.453756,192300,439070,11545680,2110755,A
3,금천구,시흥제3동,골목상권,시흥3동 우편취급국,126.899461,37.444233,191289,438014,11545690,2110739,A
4,송파구,방이2동,골목상권,송파나루역 3번,127.113516,37.511812,210222,445516,11710562,2111017,A


In [ ]:
standard.shape

(1671, 11)

In [ ]:
standard.to_csv('/content/drive/MyDrive/광진구팟/Data/상권분석/서울시 상권.csv', index = False)

In [ ]:
# 광진구만 뽑아 저장
standard_광진구 = standard[standard['시군구명'] == '광진구']
standard_광진구.head()

,시군구명,행정동명,상권_구분_코드_명,상권_코드_명,경도,위도,엑스좌표_값,와이좌표_값,행정동코드,상권_코드,상권_구분_코드
51,광진구,군자동,골목상권,세종대학교,127.074085,37.555226,206732,450331,11215730,2110149,A
60,광진구,중곡제1동,골목상권,중곡1동 마을마당,127.074513,37.564290,206769,451337,11215740,2110150,A
99,광진구,구의제1동,골목상권,광진경찰서,127.081126,37.546446,207355,449357,11215850,2110160,A
182,광진구,자양제3동,골목상권,신자초등학교,127.070534,37.534406,206420,448020,11215840,2110146,A
184,광진구,자양제2동,골목상권,자양한강도서관,127.075161,37.533746,206829,447947,11215830,2110151,A


In [ ]:
standard_광진구['행정동명'].unique()

array(['군자동', '중곡제1동', '구의제1동', '자양제3동', '자양제2동', '자양제1동', '화양동', '광장동',
       '자양제4동', '능동', '구의제2동', '중곡제4동', '구의제3동', '중곡제2동', '중곡제3동'],
      dtype=object)

In [ ]:
# 광진구 법정동 생성

def get_법정동(x):
  if "광장동" in x:
    return "광장동"
  elif "구의" in x:
    return "구의동"
  elif "군자동" in x:
    return "군자동"
  elif "능동" in x:
    return "능동"
  elif "자양" in x:
    return "자양동"
  elif "중곡" in x:
    return "중곡동"
  elif "화양" in x:
    return "화양동"

In [ ]:
standard_광진구['법정동명'] = standard_광진구['행정동명'].map(lambda x: get_법정동(x))

<ipython-input-90-21278eef35f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standard_광진구['법정동명'] = standard_광진구['행정동명'].map(lambda x: get_법정동(x))


In [ ]:
standard_광진구.reset_index(inplace = True, drop = True)
standard_광진구.head()

,시군구명,행정동명,상권_구분_코드_명,상권_코드_명,경도,위도,엑스좌표_값,와이좌표_값,행정동코드,상권_코드,상권_구분_코드,법정동명
0,광진구,군자동,골목상권,세종대학교,127.074085,37.555226,206732,450331,11215730,2110149,A,군자동
1,광진구,중곡제1동,골목상권,중곡1동 마을마당,127.074513,37.564290,206769,451337,11215740,2110150,A,중곡동
2,광진구,구의제1동,골목상권,광진경찰서,127.081126,37.546446,207355,449357,11215850,2110160,A,구의동
3,광진구,자양제3동,골목상권,신자초등학교,127.070534,37.534406,206420,448020,11215840,2110146,A,자양동
4,광진구,자양제2동,골목상권,자양한강도서관,127.075161,37.533746,206829,447947,11215830,2110151,A,자양동


In [ ]:
standard_광진구.to_csv('/content/drive/MyDrive/광진구팟/Data/상권분석/광진구 상권.csv', index = False)